In [3]:
# -*- coding: utf-8 -*-
"""
논문 종목 8개(미국 기술주) 2020-01-01 ~ 오늘까지의 OHLCV 수집 스크립트
- 무료: yfinance 사용
- auto_adjust=True 로 분할/배당 반영된 가격 사용(연구 재현에 편리)
- 각 종목별 CSV 캐싱 저장 (data/raw/{TICKER}.csv)
- 중복 인덱스/결측/열정렬 처리 포함
"""

import os
import time
from datetime import date
from typing import List, Dict

import numpy as np
import pandas as pd
import yfinance as yf

# -----------------------
# 설정
# -----------------------
TICKERS: List[str] = ["AAPL", "ADBE", "AMD", "CRM", "MSFT", "NOW", "NVDA", "ORCL"]
START_DATE = "2020-01-01"
END_DATE = date.today().strftime("%Y-%m-%d")  # 오늘
DATA_DIR = "data/raw"
RETRIES = 3
SLEEP_SEC = 2

os.makedirs(DATA_DIR, exist_ok=True)

# yfinance 전역 옵션(진행 바 끄기)
# pandas_datareader 호환은 불필요하므로 비활성화
# yf는 별도 옵션 필요 없음. download에서 progress=False 사용.

# -----------------------
# 유틸 함수
# -----------------------
def _clean_ohlcv(df: pd.DataFrame) -> pd.DataFrame:
    """
    yfinance DataFrame을 정리:
    - 열 이름 표준화(Open/High/Low/Close/Adj Close/Volume)
    - 중복 인덱스 제거
    - 정렬 및 결측 제거(모든 열 NA인 행 제거)
    - DatetimeIndex 보장 및 index name을 'Date'로 설정
    """
    # 열 이름 표준화
    rename_map = {
        "Open": "Open",
        "High": "High",
        "Low": "Low",
        "Close": "Close",
        "Adj Close": "Adj Close",
        "AdjClose": "Adj Close",  # 혹시 모를 변형
        "Volume": "Volume",
    }
    df = df.rename(columns=rename_map)

    # 필요 열 보장(없으면 생성)
    for col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]:
        if col not in df.columns:
            # Adj Close가 없을 때 Close로 대체, 나머지는 NaN 생성
            if col == "Adj Close" and "Close" in df.columns:
                df[col] = df["Close"]
            else:
                df[col] = np.nan

    # 열 순서 고정
    df = df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]]

    # 인덱스 정리
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df.index.name = "Date"
    df = df[~df.index.duplicated(keep="last")]
    df = df.sort_index()

    # 모든 열이 결측인 날은 제거
    df = df.dropna(how="all")

    return df


def fetch_ohlcv(ticker: str,
                start: str,
                end: str,
                auto_adjust: bool = True,
                retries: int = 3,
                sleep_sec: int = 2) -> pd.DataFrame:
    """
    단일 종목 OHLCV 다운로드(리트라이 포함) 후 정리 반환
    """
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            df = yf.download(
                ticker,
                start=start,
                end=end,
                auto_adjust=auto_adjust,  # 분할/배당 반영 가격
                progress=False,
                threads=True,
                interval="1d",
            )
            # 다운로드가 비어있으면 예외
            if df is None or df.empty:
                raise ValueError(f"Empty DataFrame for {ticker}")

            df = _clean_ohlcv(df)
            return df

        except Exception as e:
            last_err = e
            if attempt < retries:
                time.sleep(sleep_sec)
            else:
                raise RuntimeError(f"[{ticker}] download failed after {retries} retries: {e}") from e


def save_csv(df: pd.DataFrame, path: str) -> None:
    # 인덱스가 날짜로 저장되도록 보장
    df.to_csv(path, index=True, encoding="utf-8")


# -----------------------
# 실행
# -----------------------
def main():
    summary: Dict[str, Dict[str, int]] = {}

    for t in TICKERS:
        print(f"▶ Fetching {t} ({START_DATE} ~ {END_DATE}) ...")
        try:
            df = fetch_ohlcv(t, START_DATE, END_DATE, auto_adjust=True, retries=RETRIES, sleep_sec=SLEEP_SEC)
        except Exception as e:
            print(f"  ! Failed {t}: {e}")
            continue

        # 기본 점검: 거래일만 존재(주말/공휴일 자동 제외)
        # 결측치 점검(거래량 0인 날은 있을 수 있음)
        na_counts = df.isna().sum().to_dict()

        out_path = os.path.join(DATA_DIR, f"{t}.csv")
        try:
            save_csv(df, out_path)
            print(f"  - Saved: {out_path} (rows={len(df)})")
        except Exception as e:
            print(f"  ! Save failed {t}: {e}")
            continue

        summary[t] = {
            "rows": len(df),
            "na_Open": int(na_counts.get("Open", 0)),
            "na_High": int(na_counts.get("High", 0)),
            "na_Low": int(na_counts.get("Low", 0)),
            "na_Close": int(na_counts.get("Close", 0)),
            "na_AdjClose": int(na_counts.get("Adj Close", 0)),
            "na_Volume": int(na_counts.get("Volume", 0)),
        }

    print("\n=== Summary ===")
    for t, info in summary.items():
        print(f"{t}: rows={info['rows']}, NA(Open/High/Low/Close/Adj/Vol)="
              f"({info['na_Open']}/{info['na_High']}/{info['na_Low']}/{info['na_Close']}/{info['na_AdjClose']}/{info['na_Volume']})")


if __name__ == "__main__":
    main()


▶ Fetching AAPL (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\AAPL.csv (rows=1438)
▶ Fetching ADBE (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\ADBE.csv (rows=1438)
▶ Fetching AMD (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\AMD.csv (rows=1438)
▶ Fetching CRM (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\CRM.csv (rows=1438)
▶ Fetching MSFT (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\MSFT.csv (rows=1438)
▶ Fetching NOW (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\NOW.csv (rows=1438)
▶ Fetching NVDA (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\NVDA.csv (rows=1438)
▶ Fetching ORCL (2020-01-01 ~ 2025-09-23) ...
  - Saved: data/raw\ORCL.csv (rows=1438)

=== Summary ===
AAPL: rows=1438, NA(Open/High/Low/Close/Adj/Vol)=(0/0/0/0/0/0)
ADBE: rows=1438, NA(Open/High/Low/Close/Adj/Vol)=(0/0/0/0/0/0)
AMD: rows=1438, NA(Open/High/Low/Close/Adj/Vol)=(0/0/0/0/0/0)
CRM: rows=1438, NA(Open/High/Low/Close/Adj/Vol)=(0/0/0/0/0/0)
MSFT: rows=1438, NA(Open/High/Low/Close/Adj

In [11]:
# -*- coding: utf-8 -*-
"""
data/raw/*.csv (OHLCV) -> data/features/*_features.csv (OHLCV + 32 indicators)
- 무료: pandas, numpy만 사용 (추가 의존성 없음)
- 결측/윈도우 초반부 dropna
- 컬럼형식 통일, 인덱스=Date
"""

import os
import numpy as np
import pandas as pd

RAW_DIR = "data/raw"
FEAT_DIR = "data/features"
os.makedirs(FEAT_DIR, exist_ok=True)

TICKERS = ["AAPL", "ADBE", "AMD", "CRM", "MSFT", "NOW", "NVDA", "ORCL"]


# 기술적 지표 계산 함수들 (pandas, numpy만 사용)
def sma(series, window):
    """Simple Moving Average"""
    return series.rolling(window=window).mean()

def ema(series, window):
    """Exponential Moving Average"""
    return series.ewm(span=window).mean()

def rsi(series, window=14):
    """Relative Strength Index"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def roc(series, window):
    """Rate of Change"""
    return series.pct_change(window) * 100

def obv(close, volume):
    """On Balance Volume"""
    obv = pd.Series(index=close.index, dtype=float)
    obv.iloc[0] = volume.iloc[0]
    for i in range(1, len(close)):
        if close.iloc[i] > close.iloc[i-1]:
            obv.iloc[i] = obv.iloc[i-1] + volume.iloc[i]
        elif close.iloc[i] < close.iloc[i-1]:
            obv.iloc[i] = obv.iloc[i-1] - volume.iloc[i]
        else:
            obv.iloc[i] = obv.iloc[i-1]
    return obv

def macd(series, fast=12, slow=26, signal=9):
    """MACD"""
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal)
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

def stochastic(high, low, close, k_window=14, d_window=3):
    """Stochastic Oscillator"""
    lowest_low = low.rolling(window=k_window).min()
    highest_high = high.rolling(window=k_window).max()
    k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    d_percent = k_percent.rolling(window=d_window).mean()
    return k_percent, d_percent

def cci(high, low, close, window=20):
    """Commodity Channel Index"""
    typical_price = (high + low + close) / 3
    sma_tp = sma(typical_price, window)
    mean_deviation = typical_price.rolling(window=window).apply(
        lambda x: np.mean(np.abs(x - x.mean())), raw=True
    )
    return (typical_price - sma_tp) / (0.015 * mean_deviation)

def atr(high, low, close, window=14):
    """Average True Range"""
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return true_range.rolling(window=window).mean()

def bollinger_bands(series, window=20, std_dev=2):
    """Bollinger Bands"""
    sma_series = sma(series, window)
    std = series.rolling(window=window).std()
    upper_band = sma_series + (std * std_dev)
    lower_band = sma_series - (std * std_dev)
    return sma_series, upper_band, lower_band

def momentum(series, window=10):
    """Momentum"""
    return series - series.shift(window)


def build_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    입력 df: columns = [Open, High, Low, Close, Adj Close, Volume], index=Date
    출력 df: OHLCV + 32 indicators (총 32개 지표 컬럼 추가)
    """
    df = df.copy().sort_index()
    # 인덱스 정리
    df.index = pd.to_datetime(df.index)
    df.index.name = "Date"

    # 필요한 원시 컬럼 보장
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    if "Adj Close" not in df.columns:
        df["Adj Close"] = df["Close"]

    # ===== 여기서부터 32개 지표 생성 =====
    O, H, L, C, V = df["Open"], df["High"], df["Low"], df["Close"], df["Volume"]

    out = pd.DataFrame(index=df.index)
    # OHLCV 원본 그대로 유지
    out["Open"] = O
    out["High"] = H
    out["Low"]  = L
    out["Close"] = C
    out["Adj Close"] = df["Adj Close"]
    out["Volume"] = V

    # (1) 가격/거래량 기본 파생 4개
    out["Close_1"]   = C.shift(1)        # 1
    out["Close_inc"] = C.diff()          # 2
    out["Volume_1"]  = V.shift(1)        # 3
    out["Volume_inc"]= V.diff()          # 4

    # (2) MA/EMA 세트 (5, 10, 20) -> 12개
    for d in [5, 10, 20]:
        out[f"MA{d}"]     = sma(C, d)                   # +3 (5,10,20)
        out[f"MA{d}_1"]   = out[f"MA{d}"].shift(1)      # +3
        out[f"MA{d}_inc"] = out[f"MA{d}"].diff()        # +3
        out[f"EMA{d}"]    = ema(C, d)                   # +3
    # 여기까지 추가된 개수: 4 + 12 = 16

    # (3) RSI(12) 1개
    out["RSI12"] = rsi(C, window=12)                    # 17

    # (4) OBV 1개
    out["OBV"] = obv(C, V)                              # 18

    # (5) ROC(5,10,20) 3개
    out["ROC5"]  = roc(C, 5)                            # 19
    out["ROC10"] = roc(C, 10)                           # 20
    out["ROC20"] = roc(C, 20)                           # 21

    # (6) MACD(12,26,9) 3개
    macd_line, signal_line, histogram = macd(C, fast=12, slow=26, signal=9)
    out["MACD"]       = macd_line                       # 22
    out["MACDsignal"] = signal_line                     # 23
    out["MACDhist"]   = histogram                       # 24

    # (7) Stochastic %K(3), %D(3) 2개 (논문 표기 근사)
    k_percent, d_percent = stochastic(H, L, C, k_window=3, d_window=3)
    out["STOCHk3"] = k_percent                          # 25
    out["STOCHd3"] = d_percent                          # 26

    # (8) CCI(10) 1개
    out["CCI10"] = cci(H, L, C, window=10)              # 27

    # (9) ATR(14) 1개 (변동성) — 실무에서 자주 포함
    out["ATR14"] = atr(H, L, C, window=14)              # 28

    # (10) Bollinger Bands(20, 2.0) 3개
    bb_middle, bb_upper, bb_lower = bollinger_bands(C, window=20, std_dev=2.0)
    out["BBM20"] = bb_middle                            # 29
    out["BBU20"] = bb_upper                             # 30
    out["BBL20"] = bb_lower                             # 31

    # (11) Momentum(10) 1개
    out["MOM10"] = momentum(C, window=10)               # 32

    # ===== 지표 생성 끝 =====

    # 지표 윈도우 초반부 결측 제거
    out = out.dropna()

    # dtype 압축(선택) - 파일 용량 절감
    float_cols = out.select_dtypes(include=["float64", "float32"]).columns
    out[float_cols] = out[float_cols].astype(np.float32)

    return out


def process_all():
    for t in TICKERS:
        in_path = os.path.join(RAW_DIR, f"{t}.csv")
        if not os.path.exists(in_path):
            print(f"[skip] {in_path} not found")
            continue

        # CSV 파일 구조에 맞게 읽기 (첫 3행은 메타데이터)
        df = pd.read_csv(in_path, skiprows=3, parse_dates=[0], index_col=0)
        df.columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]
        df.index.name = "Date"
        
        feats = build_features(df)

        out_path = os.path.join(FEAT_DIR, f"{t}_features.csv")
        feats.to_csv(out_path, encoding="utf-8")
        print(f"[ok] {t}: features -> {out_path} (rows={len(feats)}, cols={feats.shape[1]})")


if __name__ == "__main__":
    process_all()


[ok] AAPL: features -> data/features\AAPL_features.csv (rows=1417, cols=38)
[ok] ADBE: features -> data/features\ADBE_features.csv (rows=1417, cols=38)
[ok] AMD: features -> data/features\AMD_features.csv (rows=1417, cols=38)
[ok] CRM: features -> data/features\CRM_features.csv (rows=1417, cols=38)
[ok] MSFT: features -> data/features\MSFT_features.csv (rows=1417, cols=38)
[ok] NOW: features -> data/features\NOW_features.csv (rows=1417, cols=38)
[ok] NVDA: features -> data/features\NVDA_features.csv (rows=1417, cols=38)
[ok] ORCL: features -> data/features\ORCL_features.csv (rows=1417, cols=38)
